# CNNLSTM


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import string
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pickle
import pandas as pd


from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

from tensorflow.keras.models import load_model
from tensorflow.keras.activations import relu
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Embedding,
    Flatten,
    Conv1D,
    MaxPooling1D,
    Add,
    Lambda,
    Dropout,
    concatenate,
    LSTM
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.backend import l2_normalize
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from IPython.display import SVG

In [3]:
def ModiData(obj_df):
    obj_df = obj_df[obj_df["label"] != 2]
    # obj_df["label"] = obj_df["label"].astype(string)
    obj_df.label = obj_df.label.replace(0, "neg")
    obj_df.label = obj_df.label.replace(1, "neg")
    obj_df.label = obj_df.label.replace(3, "pos")
    obj_df.label = obj_df.label.replace(4, "pos")
    obj_df = obj_df.fillna(0)
    # obj_df["label"] = obj_df["label"].astype(int)
    return obj_df


def clean_line(line):
    # split into tokens by white space
    tokens = line.split()
    # remove punctuation from each token
    table = str.maketrans("", "", string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words("english"))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer


def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding="post")
    return padded

In [4]:
train_df = pd.read_csv(
    "/content/drive/MyDrive/Sentiment Analysis/IMDB/train.csv", 
)

test_df = pd.read_csv(
    "/content/drive/MyDrive/Sentiment Analysis/IMDB/test.csv", 
)

train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [5]:
# train_df = ModiData(train_df)
# # dev_df = ModiData(dev_df)
# test_df = ModiData(test_df)

train_x = train_df.text.tolist()
train_y = train_df.label
# dev_x = dev_df.text.tolist()
# dev_y = dev_df.label
test_x = test_df.text.tolist()
test_y = test_df.label

max_len = 200
# tokenizer = create_tokenizer(train_x)
with open("/content/drive/MyDrive/Sentiment Analysis/tokenizer/200in_IMDB.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
train_x = encode_text(tokenizer, train_x, max_len)
# dev_x = encode_text(tokenizer, dev_x, max_len)
test_x = encode_text(tokenizer, test_x, max_len)

le = LabelEncoder()
train_y = le.fit_transform(train_y).reshape(-1, 1)
# dev_y = le.transform(dev_y).reshape(-1, 1)
test_y = le.transform(test_y).reshape(-1, 1)

## Model

In [31]:
def get_embeddings_layer(name, max_len, trainable):
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=50,
        input_length=max_len,
        # weights=[embeddings_matrix],
        trainable=trainable,
        name=name,
    )
    return embedding_layer


def get_conv_pool(x_input, max_len, sufix, n_grams=[3, 4, 5], feature_maps=100):
    branches = []
    for n in n_grams:
        branch = Conv1D(
            filters=feature_maps,
            kernel_size=n,
            activation=relu,
            name="Conv_" + sufix + "_" + str(n),
        )(x_input)
        branch = MaxPooling1D(
            pool_size=2,
            strides=None,
            padding="valid",
            name="MaxPooling_" + sufix + "_" + str(n),
        )(branch)
        branches.append(branch)
    return branches


def get_cnn_rand(max_len):
    embeddings_layer_channel_1 = get_embeddings_layer(
        "embeddings_layer_dynamic", max_len, trainable=True
    )

    input_dynamic = Input(shape=(max_len,), dtype="int32", name="input_dynamic")
    x = embeddings_layer_channel_1(input_dynamic)
    branches_dynamic = get_conv_pool(x, max_len, "dynamic")
    z_dynamic = concatenate(branches_dynamic, axis=1)

    layer = LSTM(128,return_sequences=True)(z_dynamic)
    layer = LSTM(128)(layer)

    z = Dropout(0.5)(layer)
    z = layer = Dense(20, activation="relu", name="FC1")(z)
    # z = Dropout(0.1)(z)
    o = Dense(1, activation="sigmoid", name="output")(z)

    model = Model(inputs=input_dynamic, outputs=o)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

    return model

In [37]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("val_loss") <= 0.34:
            print("\nReached 0.32 loss")
            self.model.stop_training = True


beststop = myCallback()
earlystop = EarlyStopping(monitor="val_loss", min_delta=0.0001)
checkpointer = ModelCheckpoint(filepath="./weights.hdf5", verbose=1)

In [38]:
def TrainModel(num):
    while num > 0:
        model = get_cnn_rand(max_len)
        # model.summary()
        history = model.fit(
            x=train_x,
            y=train_y,
            batch_size=200,
            epochs=8,
            validation_data=(test_x, test_y),
            callbacks=[beststop],
        )
        if history.history["val_acc"][-1] > 0.86:
            save_name = (
                "/content/drive/MyDrive/Sentiment Analysis/Model/CNNLSTM_IMDB/"
                + str(history.history["val_loss"][-1])[2:6]
                + "_"
                + str(history.history["val_acc"][-1])[2:6]
                + ".hdf5"
            )
            num -= 1
            model.save(save_name)

In [39]:
TrainModel(10)

Epoch 1/8
125/125 [==============================] - 17s 117ms/step - loss: 0.6567 - acc: 0.5726 - val_loss: 0.3760 - val_acc: 0.8496
Epoch 2/8
125/125 [==============================] - 13s 108ms/step - loss: 0.2640 - acc: 0.8999 - val_loss: 0.3179 - val_acc: 0.8683

Reached 0.32 loss
Epoch 1/8
125/125 [==============================] - 17s 114ms/step - loss: 0.6803 - acc: 0.5201 - val_loss: 0.4186 - val_acc: 0.8178
Epoch 2/8
125/125 [==============================] - 13s 107ms/step - loss: 0.2816 - acc: 0.8933 - val_loss: 0.3189 - val_acc: 0.8695

Reached 0.32 loss
Epoch 1/8
125/125 [==============================] - 18s 124ms/step - loss: 0.6618 - acc: 0.5531 - val_loss: 0.4000 - val_acc: 0.8374
Epoch 2/8
125/125 [==============================] - 14s 112ms/step - loss: 0.2772 - acc: 0.8998 - val_loss: 0.3532 - val_acc: 0.8583
Epoch 3/8
125/125 [==============================] - 14s 111ms/step - loss: 0.1127 - acc: 0.9625 - val_loss: 0.4114 - val_acc: 0.8548
Epoch 4/8
125/125 [=====

In [35]:
# del model
model = get_cnn_rand(max_len)
model.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_dynamic (InputLayer)      [(None, 200)]        0                                            
__________________________________________________________________________________________________
embeddings_layer_dynamic (Embed (None, 200, 50)      4429150     input_dynamic[0][0]              
__________________________________________________________________________________________________
Conv_dynamic_3 (Conv1D)         (None, 198, 100)     15100       embeddings_layer_dynamic[0][0]   
__________________________________________________________________________________________________
Conv_dynamic_4 (Conv1D)         (None, 197, 100)     20100       embeddings_layer_dynamic[0][0]   
___________________________________________________________________________________________

In [ ]:
history = model.fit(
            x=train_x,
            y=train_y,
            batch_size=200,
            epochs=20,
            validation_data=(test_x, test_y),
            # callbacks=[earlystop],
        )

Epoch 1/20
125/125 [==============================] - 12s 74ms/step - loss: 0.6897 - acc: 0.5170 - val_loss: 0.4885 - val_acc: 0.8022
Epoch 2/20
125/125 [==============================] - 9s 69ms/step - loss: 0.4004 - acc: 0.8446 - val_loss: 0.3068 - val_acc: 0.8709
Epoch 3/20
125/125 [==============================] - 9s 68ms/step - loss: 0.2246 - acc: 0.9197 - val_loss: 0.4458 - val_acc: 0.8310
Epoch 4/20
 63/125 [==============>...............] - ETA: 3s - loss: 0.1922 - acc: 0.9408

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.models import load_model

# 保存模型
# model.save("10in_32unit_temp_res.h5")
# del model  # deletes the existing model
# 导入已经训练好的模型
# model = load_model("my_model.h5")
## 保存训练好的Tokenizer，和导入
import pickle

# saving
with open("/content/drive/MyDrive/Sentiment Analysis/tokenizer/200in_IMDB.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
loss, accuracy = model.evaluate(x=[test_x, test_x], y=test_y, verbose=0)
print(loss)
print(accuracy)

### google

In [ ]:
def load_google():
    Word2VecModel = gensim.models.KeyedVectors.load_word2vec_format(
        r"E:\CS\MLT\GoogleNews-vectors-negative300.bin", binary=True
    )

    vocab_list = [word for word, Vocab in Word2VecModel.wv.vocab.items()]  # 存储 所有的 词语

    # word_index = {" ": 0}  # 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
    embeddings_index = {}  # 初始化`[word : vector]`字典

    for i in range(len(vocab_list)):
        # print(i)
        word = vocab_list[i]  # 每个词语
        # word_index[word] = i + 1 # 词语：序号
        embeddings_index[word] = Word2VecModel.wv[word]  # 词语：词向量
        # embeddings_matrix[i + 1] = Word2VecModel.wv[word]  # 词向量矩阵
    return embeddings_index


def load_fasttext_embeddings():
    glove_dir = r"E:\CS\MLT\glove.6B"
    embeddings_index = {}
    f = open(os.path.join(glove_dir, "glove.6B.50d.txt"), encoding="utf-8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
    f.close()
    print("Found %s word vectors." % len(embeddings_index))
    return embeddings_index


def create_embeddings_matrix(embeddings_index, vocabulary, embedding_dim=100):
    embeddings_matrix = np.random.rand(len(vocabulary) + 1, embedding_dim)
    for i, word in enumerate(vocabulary):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector
    print("Matrix shape: {}".format(embeddings_matrix.shape))
    return embeddings_matrix


embeddings_index = load_google()
embeddings_matrix = create_embeddings_matrix(embeddings_index, word_index, 300)

In [ ]:
import gensim

Word2VecModel = gensim.models.KeyedVectors.load_word2vec_format(
    r"E:\CS\MLT\GoogleNews-vectors-negative300.bin", binary=True
)

vocab_list = [word for word, Vocab in Word2VecModel.wv.vocab.items()]  # 存储 所有的 词语

word_index = {" ": 0}  # 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
word_vector = {}  # 初始化`[word : vector]`字典

for i in range(len(vocab_list)):
    # print(i)
    word = vocab_list[i]  # 每个词语
    #     word_index[word] = i + 1 # 词语：序号
    #     word_vector[word] = Word2VecModel.wv[word] # 词语：词向量
    embeddings_matrix[i + 1] = Word2VecModel.wv[word]  # 词向量矩阵